## DPP environment

In [1]:
import numpy as np
from numpy.linalg import inv


def decap_placement(
    n,
    m,
    raw_pdn,
    pi,
    probing_port,
    freq_pts,
    fpath="data/01nF_decap.npy",
):
    num_decap = np.size(pi)
    probe = probing_port
    z1 = raw_pdn

    with open(fpath, "rb") as f:
        decap = np.load(f)
    decap = decap.reshape(-1)
    z2 = np.zeros((freq_pts, num_decap, num_decap))

    ##### OLD ########
    # qIndx = []
    # for i in range(num_decap):
    #     z2[:, i, i] = np.abs(decap)
    #     qIndx.append(i)
    # pIndx = pi.astype(int)
    ##### OLD ########

    qIndx = np.arange(num_decap)
    z2[:, qIndx, qIndx] = np.abs(decap)[:, None].repeat(z2[:, qIndx, qIndx].shape[-1], axis=-1)
    pIndx = pi.astype(int)

    # pIndx : index of ports in z1 for connecting
    # qIndx : index of ports in z2 for connecting

    aIndx = np.arange(len(z1[0]))

    aIndx = np.delete(aIndx, pIndx)

    z1aa = z1[:, aIndx, :][:, :, aIndx]
    z1ap = z1[:, aIndx, :][:, :, pIndx]
    z1pa = z1[:, pIndx, :][:, :, aIndx]
    z1pp = z1[:, pIndx, :][:, :, pIndx]
    z2qq = z2[:, qIndx, :][:, :, qIndx]

    zout = z1aa - np.matmul(np.matmul(z1ap, inv(z1pp + z2qq)), z1pa)

    ###### OLD ########
    # for i in range(n * m):
    #     if i in pi:

    #         if i < probing_port:
    #             probe = probe - 1
    ###### OLD ########

    # New
    idx = np.arange(n * m)
    mask = np.zeros(n * m).astype(bool)
    mask[pi] = True
    mask = mask & (idx < probing_port)
    probe -= mask.sum().item()

    zout = zout[:, probe, probe]
    return zout


def decap_model(
    z_initial, z_final, N_freq, fpath="data/freq_201.npy"
):

    impedance_gap = np.zeros(N_freq)

    with open(fpath, "rb") as f:
        freq = np.load(f)

    ###### OLD ########
    # reward = 0
    # for i in range(N_freq):
    #     impedance_gap[i] = z_initial[i] - z_final[i]
    #     reward = reward + (impedance_gap[i] * 1000000000 / freq[i])
    ###### OLD ########

    impedance_gap = z_initial - z_final
    reward = np.sum(impedance_gap * 1000000000 / freq)

    reward = reward / 10
    return reward


def initial_impedance(n, m, raw_pdn, probe):

    zout = raw_pdn[:, probe, probe]

    return zout


def decap_sim(
    probe,
    solution,
    keepout=None,
    N=10,
    N_freq=201,
    fpath="data/10x10_pkg_chip.npy",
):

    with open(fpath, "rb") as f:
        raw_pdn = np.load(f)
    solution = np.array(solution)

    assert len(solution) == len(
        np.unique(solution)
    ), "An Element of Decap Sequence must be Unique"

    if keepout is not None:
        keepout = np.array(keepout)
        intersect = np.intersect1d(solution, keepout)
        assert len(intersect) == 0, "Decap must be not placed at the keepout region"

    z_initial = initial_impedance(N, N, raw_pdn, probe)
    z_initial = np.abs(z_initial)
    z_final = decap_placement(N, N, raw_pdn, solution, probe, N_freq)
    z_final = np.abs(z_final)
    reward = decap_model(z_initial, z_final, N_freq)

    return reward

reward = decap_sim(probe = 23, solution = [1,5,7,21], keepout = [2,3,10])


Test reward: `6.684142842810994`


In [2]:
reward = decap_sim(probe = 23, solution = [1,5,7,21], keepout = [2,3,10])
print(reward)

6.684142842811002


In [3]:
def generate_init_conditions(m=10, n=10, num_keepout_min=1, num_keepout_max=50):
    # Create a list of observations on a grid
    # n: number of rows and columns
    # num_keepout: number of keepout regions
    # return: a list of observations

    # Create a list of observations on a grid
    locs = np.meshgrid(np.arange(m), np.arange(n))
    locs = np.stack(locs, axis=-1).reshape(-1, 2)
    # normalize the locations by the number of rows and columns
    locs = locs / np.array([m, n])

    # Create available mask
    available = np.ones((m * n), dtype=np.bool)

    # Sample probe location from m*n
    probe = np.random.choice(np.arange(m * n))
    available[probe] = False

    # Sample keepout locations from m*n except probe
    num_keepout = np.random.randint(num_keepout_min, num_keepout_max)
    keepout = np.random.choice(np.arange(m * n), num_keepout, replace=False)
    available[keepout] = False

    return locs, available, probe


locs, available, probe = generate_init_conditions()
print(locs, available, probe)

[[0.  0. ]
 [0.1 0. ]
 [0.2 0. ]
 [0.3 0. ]
 [0.4 0. ]
 [0.5 0. ]
 [0.6 0. ]
 [0.7 0. ]
 [0.8 0. ]
 [0.9 0. ]
 [0.  0.1]
 [0.1 0.1]
 [0.2 0.1]
 [0.3 0.1]
 [0.4 0.1]
 [0.5 0.1]
 [0.6 0.1]
 [0.7 0.1]
 [0.8 0.1]
 [0.9 0.1]
 [0.  0.2]
 [0.1 0.2]
 [0.2 0.2]
 [0.3 0.2]
 [0.4 0.2]
 [0.5 0.2]
 [0.6 0.2]
 [0.7 0.2]
 [0.8 0.2]
 [0.9 0.2]
 [0.  0.3]
 [0.1 0.3]
 [0.2 0.3]
 [0.3 0.3]
 [0.4 0.3]
 [0.5 0.3]
 [0.6 0.3]
 [0.7 0.3]
 [0.8 0.3]
 [0.9 0.3]
 [0.  0.4]
 [0.1 0.4]
 [0.2 0.4]
 [0.3 0.4]
 [0.4 0.4]
 [0.5 0.4]
 [0.6 0.4]
 [0.7 0.4]
 [0.8 0.4]
 [0.9 0.4]
 [0.  0.5]
 [0.1 0.5]
 [0.2 0.5]
 [0.3 0.5]
 [0.4 0.5]
 [0.5 0.5]
 [0.6 0.5]
 [0.7 0.5]
 [0.8 0.5]
 [0.9 0.5]
 [0.  0.6]
 [0.1 0.6]
 [0.2 0.6]
 [0.3 0.6]
 [0.4 0.6]
 [0.5 0.6]
 [0.6 0.6]
 [0.7 0.6]
 [0.8 0.6]
 [0.9 0.6]
 [0.  0.7]
 [0.1 0.7]
 [0.2 0.7]
 [0.3 0.7]
 [0.4 0.7]
 [0.5 0.7]
 [0.6 0.7]
 [0.7 0.7]
 [0.8 0.7]
 [0.9 0.7]
 [0.  0.8]
 [0.1 0.8]
 [0.2 0.8]
 [0.3 0.8]
 [0.4 0.8]
 [0.5 0.8]
 [0.6 0.8]
 [0.7 0.8]
 [0.8 0.8]
 [0.9 0.8]
 [0.  0.9]

/tmp/ipykernel_1272563/925642795.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  available = np.ones((m * n), dtype=np.bool)


In [4]:
import numpy as np
from numpy.linalg import inv


def decap_placement(
    n,
    m,
    raw_pdn,
    pi,
    probing_port,
    freq_pts,
    fpath="data/01nF_decap.npy",
):
    num_decap = np.size(pi)
    probe = probing_port
    z1 = raw_pdn

    with open(fpath, "rb") as f:
        decap = np.load(f)
    decap = decap.reshape(-1)
    z2 = np.zeros((freq_pts, num_decap, num_decap))


    qIndx = np.arange(num_decap)
    z2[:, qIndx, qIndx] = np.abs(decap)[:, None].repeat(z2[:, qIndx, qIndx].shape[-1], axis=-1)
    pIndx = pi.astype(int)

    # pIndx : index of ports in z1 for connecting
    # qIndx : index of ports in z2 for connecting

    aIndx = np.arange(len(z1[0]))

    aIndx = np.delete(aIndx, pIndx)

    z1aa = z1[:, aIndx, :][:, :, aIndx]
    z1ap = z1[:, aIndx, :][:, :, pIndx]
    z1pa = z1[:, pIndx, :][:, :, aIndx]
    z1pp = z1[:, pIndx, :][:, :, pIndx]
    z2qq = z2[:, qIndx, :][:, :, qIndx]

    zout = z1aa - np.matmul(np.matmul(z1ap, inv(z1pp + z2qq)), z1pa)

    # New
    idx = np.arange(n * m)
    mask = np.zeros(n * m).astype(bool)
    mask[pi] = True
    mask = mask & (idx < probing_port)
    probe -= mask.sum().item()

    zout = zout[:, probe, probe]
    return zout


def decap_model(
    z_initial, z_final, N_freq, fpath="data/freq_201.npy"
):

    impedance_gap = np.zeros(N_freq)

    with open(fpath, "rb") as f:
        freq = np.load(f)
        
    impedance_gap = z_initial - z_final
    reward = np.sum(impedance_gap * 1000000000 / freq)

    reward = reward / 10
    return reward


def initial_impedance(n, m, raw_pdn, probe):

    zout = raw_pdn[:, probe, probe]

    return zout


def decap_sim(
    probe,
    solution,
    keepout=None,
    N=10,
    N_freq=201,
    fpath="data/10x10_pkg_chip.npy",
):

    with open(fpath, "rb") as f:
        raw_pdn = np.load(f)
    solution = np.array(solution)

    assert len(solution) == len(
        np.unique(solution)
    ), "An Element of Decap Sequence must be Unique"

    if keepout is not None:
        keepout = np.array(keepout)
        intersect = np.intersect1d(solution, keepout)
        assert len(intersect) == 0, "Decap must be not placed at the keepout region"

    z_initial = initial_impedance(N, N, raw_pdn, probe)
    z_initial = np.abs(z_initial)
    z_final = decap_placement(N, N, raw_pdn, solution, probe, N_freq)
    z_final = np.abs(z_final)
    reward = decap_model(z_initial, z_final, N_freq)

    return reward

In [5]:
reward = decap_sim(probe = 23, solution = [1,5,7,21], keepout = [2,3,10])
print(reward)

6.684142842811002


In [6]:
import torch


def decap_placement(n, m, raw_pdn, pi, probing_port, freq_pts, fpath="data/01nF_decap.npy"):
    num_decap = torch.numel(pi)
    probe = probing_port
    z1 = raw_pdn

    with open(fpath, "rb") as f:
        decap = torch.from_numpy(np.load(f)).to(torch.complex64)
    decap = decap.reshape(-1)
    z2 = torch.zeros((freq_pts, num_decap, num_decap), dtype=torch.float32)

    qIndx = torch.arange(num_decap)

    z2[:, qIndx, qIndx] = torch.abs(decap)[:, None].repeat_interleave(z2[:, qIndx, qIndx].shape[-1], dim=-1)
    pIndx = pi.long()

    aIndx = torch.arange(len(z1[0]))
    aIndx = torch.tensor(list(set(aIndx.tolist()) - set(pIndx.tolist())))

    z1aa = z1[:, aIndx, :][:, :, aIndx]
    z1ap = z1[:, aIndx, :][:, :, pIndx]
    z1pa = z1[:, pIndx, :][:, :, aIndx]
    z1pp = z1[:, pIndx, :][:, :, pIndx]
    z2qq = z2[:, qIndx, :][:, :, qIndx]

    zout = z1aa - torch.matmul(torch.matmul(z1ap, torch.inverse(z1pp + z2qq)), z1pa)

    idx = torch.arange(n * m)
    mask = torch.zeros(n * m).bool()
    mask[pi] = True
    mask = mask & (idx < probing_port)
    probe -= mask.sum().item()

    zout = zout[:, probe, probe]
    return zout


def decap_model(z_initial, z_final, N_freq, fpath="data/freq_201.npy"):
    impedance_gap = torch.zeros(N_freq)

    with open(fpath, "rb") as f:
        freq = torch.from_numpy(np.load(f))
        
    impedance_gap = z_initial - z_final
    reward = torch.sum(impedance_gap * 1000000000 / freq)

    reward = reward / 10
    return reward


def initial_impedance(n, m, raw_pdn, probe):
    zout = raw_pdn[:, probe, probe]
    return zout


def decap_sim(probe, solution, keepout=None, N=10, N_freq=201, fpath="data/10x10_pkg_chip.npy"):
    with open(fpath, "rb") as f:
        raw_pdn = torch.from_numpy(np.load(f))
    solution = torch.tensor(solution)

    assert len(solution) == len(torch.unique(solution)), "An Element of Decap Sequence must be Unique"

    if keepout is not None:
        keepout = torch.tensor(keepout)
        intersect = torch.tensor(list(set(solution.tolist()) & set(keepout.tolist())))
        assert len(intersect) == 0, "Decap must be not placed at the keepout region"

    z_initial = initial_impedance(N, N, raw_pdn, probe)
    z_initial = torch.abs(z_initial)
    z_final = decap_placement(N, N, raw_pdn, solution, probe, N_freq)
    z_final = torch.abs(z_final)
    reward = decap_model(z_initial, z_final, N_freq)

    return reward


In [7]:
reward = decap_sim(probe = 23, solution = [1,5,7,21], keepout = [2,3,10])
print(reward)

tensor(6.6841, dtype=torch.float64)


## Passing tensors instead

In [8]:
import torch


class DecapSimulator:

    def __init__(self, num=10, num_freq=201, chip_fpath="data/10x10_pkg_chip.npy", decap_fpath="data/01nF_decap.npy", freq_fpath="data/freq_201.npy"):

        self.num = num
        self.num_freq = num_freq

        with open(freq_fpath, "rb") as f:
            self.freq = torch.from_numpy(np.load(f))
        
        with open(chip_fpath, "rb") as f:
            self.raw_pdn = torch.from_numpy(np.load(f))

        with open(decap_fpath, "rb") as f:
            self.decap = torch.from_numpy(np.load(f)).to(torch.complex64)
    
    def decap_placement(self, pi, probing_port):
        n = m = self.num # columns and rows
        num_decap = torch.numel(pi)
        probe = probing_port
        z1 = self.raw_pdn
        
        decap = self.decap.reshape(-1)
        z2 = torch.zeros((self.num_freq, num_decap, num_decap), dtype=torch.float32)

        qIndx = torch.arange(num_decap)

        z2[:, qIndx, qIndx] = torch.abs(decap)[:, None].repeat_interleave(z2[:, qIndx, qIndx].shape[-1], dim=-1)
        pIndx = pi.long()

        ####### OLD CODE #######
        aIndx = torch.arange(len(z1[0]))
        aIndx = torch.tensor(list(set(aIndx.tolist()) - set(pIndx.tolist())))
        ####### OLD CODE #######

        # batch_size = 1
        # aIndx_batch = torch.arange(len(z1[0])).unsqueeze(0).repeat(batch_size, 1)
        # pIndx_batch = pIndx.unsqueeze(0).repeat(batch_size, 1)
        # mask_batch = torch.ones_like(aIndx_batch).scatter(1, pIndx_batch, 0)
        # aIndx_batch = torch.index_select(aIndx_batch, 1, mask_batch.nonzero()[:, 1])
        # aIndx = aIndx_batch[0]

        # # batched selection
        # aIndx = aIndx.unsqueeze(0).repeat(batch_size, 1)
        # pIndx = pIndx.unsqueeze(0).repeat(batch_size, 1)

        # z1aa = torch.gather(z1, 1, aIndx.unsqueeze(2).repeat(1, 1, len(aIndx)))
        # num_indices = aIndx.shape[-1]
        # z1aa = torch.gather(z1, 1, aIndx.unsqueeze(-1).repeat(1, 1, num_indices))
        # print(z1aa.shape)
        # z1aa = torch.gather(z1aa, 2, aIndx.unsqueeze(1).repeat(1, num_indices, 1))
        # print(z1aa.shape)

        # aIndx = aIndx[0]
        # pIndx = pIndx[0]


        z1aa = z1[:, aIndx, :][:, :, aIndx]
        z1ap = z1[:, aIndx, :][:, :, pIndx]
        z1pa = z1[:, pIndx, :][:, :, aIndx]
        z1pp = z1[:, pIndx, :][:, :, pIndx]
        z2qq = z2[:, qIndx, :][:, :, qIndx]

        zout = z1aa - torch.matmul(torch.matmul(z1ap, torch.inverse(z1pp + z2qq)), z1pa)

        idx = torch.arange(n * m)
        mask = torch.zeros(n * m).bool()
        mask[pi] = True
        mask = mask & (idx < probing_port)
        probe -= mask.sum().item()

        zout = zout[:, probe, probe]
        return zout

    def decap_model(self, z_initial, z_final):
        impedance_gap = torch.zeros(self.num_freq)

        impedance_gap = z_initial - z_final
        reward = torch.sum(impedance_gap * 1000000000 / self.freq)

        reward = reward / 10
        return reward

    def initial_impedance(self, probe):
        zout = self.raw_pdn[:, probe, probe]
        return zout

    def decap_sim(self, probe, solution, keepout=None):
        probe = probe.item()

        assert len(solution) == len(torch.unique(solution)), "An Element of Decap Sequence must be Unique"

        if keepout is not None:
            keepout = torch.tensor(keepout)
            intersect = torch.tensor(list(set(solution.tolist()) & set(keepout.tolist())))
            assert len(intersect) == 0, "Decap must be not placed at the keepout region"

        z_initial = self.initial_impedance(probe)
        z_initial = torch.abs(z_initial)
        z_final = self.decap_placement(solution, probe)
        z_final = torch.abs(z_final)
        reward = self.decap_model(z_initial, z_final)

        return reward


env = DecapSimulator()

probe = torch.Tensor([23]).long()
solution = torch.Tensor([1,5,7,21]).long()
keepout = torch.Tensor([2,3,10]).long()

reward = env.decap_sim(probe, solution, keepout)
print(reward)

tensor(6.6841, dtype=torch.float64)


/tmp/ipykernel_1272563/800108083.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  keepout = torch.tensor(keepout)


## Main

---

In [56]:
import torch


class DecapSimulator:

    def __init__(self, num=10, num_freq=201, chip_fpath="data/10x10_pkg_chip.npy", decap_fpath="data/01nF_decap.npy", freq_fpath="data/freq_201.npy"):

        self.size = num
        self.num_freq = num_freq

        with open(freq_fpath, "rb") as f:
            self.freq = torch.from_numpy(np.load(f))
        
        with open(chip_fpath, "rb") as f:
            self.raw_pdn = torch.from_numpy(np.load(f))

        with open(decap_fpath, "rb") as f:
            self.decap = torch.from_numpy(np.load(f)).to(torch.complex64)
    
    def decap_placement(self, pi, probe):
        n = m = self.size # columns and rows
        num_decap = torch.numel(pi)
        z1 = self.raw_pdn
        
        decap = self.decap.reshape(-1)
        z2 = torch.zeros((self.num_freq, num_decap, num_decap), dtype=torch.float32)

        qIndx = torch.arange(num_decap)

        z2[:, qIndx, qIndx] = torch.abs(decap)[:, None].repeat_interleave(z2[:, qIndx, qIndx].shape[-1], dim=-1)
        pIndx = pi.long()

        aIndx = torch.arange(len(z1[0]))
        aIndx = torch.tensor(list(set(aIndx.tolist()) - set(pIndx.tolist())))

        z1aa = z1[:, aIndx, :][:, :, aIndx]
        z1ap = z1[:, aIndx, :][:, :, pIndx]
        z1pa = z1[:, pIndx, :][:, :, aIndx]
        z1pp = z1[:, pIndx, :][:, :, pIndx]
        z2qq = z2[:, qIndx, :][:, :, qIndx]

        zout = z1aa - torch.matmul(torch.matmul(z1ap, torch.inverse(z1pp + z2qq)), z1pa)

        idx = torch.arange(n * m)
        mask = torch.zeros(n * m).bool()
        mask[pi] = True
        mask = mask & (idx < probe)
        probe -= mask.sum().item()

        zout = zout[:, probe, probe]
        return zout

    def decap_model(self, z_initial, z_final):
        impedance_gap = torch.zeros(self.num_freq)

        impedance_gap = z_initial - z_final
        reward = torch.sum(impedance_gap * 1000000000 / self.freq)

        reward = reward / 10
        return reward

    def initial_impedance(self, probe):
        zout = self.raw_pdn[:, probe, probe]
        return zout

    def decap_sim(self, probe, solution, keepout=None):
        probe = probe.item()

        print(solution)
        print(probe)

        assert len(solution) == len(torch.unique(solution)), "An Element of Decap Sequence must be Unique"

        if keepout is not None:
            keepout = torch.tensor(keepout)
            intersect = torch.tensor(list(set(solution.tolist()) & set(keepout.tolist())))
            assert len(intersect) == 0, "Decap must be not placed at the keepout region"

        z_initial = self.initial_impedance(probe)
        z_initial = torch.abs(z_initial)
        z_final = self.decap_placement(solution, probe)
        z_final = torch.abs(z_final)
        reward = self.decap_model(z_initial, z_final)

        return reward

env = DecapSimulator()

probe = torch.Tensor([23]).long()
solution = torch.Tensor([1,5,7,21]).long()
keepout = torch.Tensor([2,3,10]).long()

reward = env.decap_sim(probe, solution, keepout)
print(reward)


# Batched
probe = probe.repeat(32, 1)
solution = solution.repeat(32, 1)
keepout = keepout.repeat(32, 1)

rewards = [r for r in map(env.decap_sim, probe, solution, keepout)]

tensor([ 1,  5,  7, 21])
23
tensor(6.6841, dtype=torch.float64)
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23
tensor([ 1,  5,  7, 21])
23


/tmp/ipykernel_1272563/987756417.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  keepout = torch.tensor(keepout)


In [53]:
print(rewards)

[tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), tensor(6.6841, dtype=torch.float64), 

In [50]:
import torch

def generate_init_conditions(batch_size, m=10, n=10, num_keepout_min=1, num_keepout_max=50):
    # Create a list of observations on a grid
    # n: number of rows and columns
    # num_keepout: number of keepout regions
    # return: a list of observations

    # Create a list of observations on a grid
    batch_size = [batch_size] if isinstance(batch_size, int) else batch_size

    locs = torch.meshgrid(torch.arange(m), torch.arange(n))
    locs = torch.stack(locs, dim=-1).reshape(-1, 2)
    # normalize the locations by the number of rows and columns
    locs = locs / torch.tensor([m, n], dtype=torch.float)

    # Create available mask
    available = torch.ones((*batch_size, m * n), dtype=torch.bool)

    # Sample probe location from m*n
    probe = torch.randint(m * n, size=(*batch_size, 1))
    available.scatter_(1, probe, False)

    # Sample keepout locations from m*n except probe
    num_keepout = torch.randint(num_keepout_min, num_keepout_max, size=(*batch_size, 1))

    print(num_keepout.shape, available.shape)
    keepouts = [torch.randperm(m * n)[:k] for k in num_keepout]
    for i, (a, k) in enumerate(zip(available, keepouts)):
        available[i] = a.scatter(0, k, False)
    # num_keepout = torch.randint(num_keepout_min, num_keepout_max, size=(batch_size, 1))
    # keepout_mask = torch.zeros((batch_size, m * n), dtype=torch.bool)

    # for b in range(batch_size):
    #     keepouts = torch.randperm(m * n)[:num_keepout[b]]
    #     keepout_mask[b, keepouts] = True

    # available.masked_fill_(keepout_mask, False)

    return locs.unsqueeze(0).repeat(*batch_size, 1, 1), available, probe


batch_size = 5
locs, available, probe = generate_init_conditions(batch_size)
print(locs.shape, available.shape, probe.shape)

torch.Size([5, 1]) torch.Size([5, 100])
torch.Size([5, 100, 2]) torch.Size([5, 100]) torch.Size([5, 1])


In [42]:
# 
torch.count_nonzero(available, dim=1)

tensor([81, 86, 63, 91, 63])

In [11]:
rewards = [r for r in map(env.decap_sim, probe, solution, keepout)]

/tmp/ipykernel_1272563/2210145885.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  keepout = torch.tensor(keepout)


## Batched environment

In [51]:
import torch
import numpy as np

class DecapSimulator:
    def __init__(self, num=10, num_freq=201, chip_fpath="data/10x10_pkg_chip.npy", decap_fpath="data/01nF_decap.npy", freq_fpath="data/freq_201.npy"):
        self.num = num
        self.num_freq = num_freq

        with open(freq_fpath, "rb") as f:
            self.freq = torch.from_numpy(np.load(f))
        
        with open(chip_fpath, "rb") as f:
            self.raw_pdn = torch.from_numpy(np.load(f))

        with open(decap_fpath, "rb") as f:
            self.decap = torch.from_numpy(np.load(f)).to(torch.complex64)
    
    def decap_placement(self, pi, probe):
        
        batch_size = pi.shape[:-1]

        n = m = self.num # columns and rows
        num_decap = pi.shape[-1] # we consider the num_decap the same in the batch
        z1 = self.raw_pdn
        
        decap = self.decap.reshape(-1)
        z2 = torch.zeros((self.num_freq, num_decap, num_decap), dtype=torch.float32)

        qIndx = torch.arange(num_decap)

        z2[:, qIndx, qIndx] = torch.abs(decap)[:, None].repeat_interleave(z2[:, qIndx, qIndx].shape[-1], dim=-1)
        pIndx = pi.long()
        
        aIndx = torch.arange(len(z1[0]))[None].expand(*batch_size, -1)
        aIndx = torch.tensor(list(set(aIndx.tolist()) - set(pIndx.tolist())))

        z1aa = z1[:, aIndx, :][:, :, aIndx]
        z1ap = z1[:, aIndx, :][:, :, pIndx]
        z1pa = z1[:, pIndx, :][:, :, aIndx]
        z1pp = z1[:, pIndx, :][:, :, pIndx]
        z2qq = z2[:, :, qIndx, :][:, :, :, qIndx]

        zout = z1aa - torch.matmul(torch.matmul(z1ap, torch.inverse(z1pp + z2qq)), z1pa)

        idx = torch.arange(n * m)
        mask = torch.zeros(batch_size, n * m).bool()
        mask[:, pi] = True
        mask = mask & (idx < probe)
        probe -= mask.sum(dim=1)

        zout = zout[torch.arange(batch_size), :, probe, probe]
        return zout

    def decap_model(self, z_initial, z_final):
        impedance_gap = torch.zeros_like(z_initial)

        impedance_gap = z_initial - z_final
        reward = torch.sum(impedance_gap * 1000000000 / self.freq, dim=1)

        reward = reward / 10
        return reward

    def initial_impedance(self, probe):
        zout = self.raw_pdn[:, probe, probe]
        return zout

    def decap_sim(self, probe, solution, keepout=None):
        
        # if keepout is not None:
        #     intersect = torch.tensor(list(set(solution.tolist()) & set(keepout.tolist())))
        #     assert len(intersect) == 0, "Decap must be not placed at the keepout region"

        z_initial = self.initial_impedance(probe).squeeze().swapaxes(0,1)
        z_initial = torch.abs(z_initial)
        z_final = self.decap_placement(solution, probe)
        z_final = torch.abs(z_final)
        reward = self.decap_model(z_initial, z_final)

        return reward
    
env = DecapSimulator()

probe = torch.Tensor([[23]]).long().repeat(32,1)
solution = torch.Tensor([[1, 5, 7, 21]]).long().repeat(32,1)
keepout = torch.Tensor([[2, 3, 10]]).long().repeat(32,1)

# Get reward for the batched input
reward = env.decap_sim(probe, solution, keepout)
print(reward)



TypeError: unhashable type: 'list'

In [ ]:
probe = torch.Tensor([23]).long()
solution = torch.Tensor([1,5,7,21]).long()
keepout = torch.Tensor([2,3,10]).long()


# Make them into a mask of 100 elements
mask = torch.zeros(100).bool()
probe = torch.scatter(mask, 0, probe, 1)
solution = torch.scatter(mask, 0, solution, 1)
keepout = torch.scatter(mask, 0, keepout, 1)

print(probe)

In [ ]:
# Get index of True element
probe = torch.nonzero(probe)
print(probe)

tensor([[23]])
